# Huggingface Sagemaker-sdk - Run a batch transform inference job with 🤗 Transformers


1. [Introduction](#Introduction)  
2. [Run Batch Transform after training a model](#Run-Batch-Transform-after-training-a-model)  
3. [Run Batch Transform Inference Job with a fine-tuned model using `jsonl`](#Run-Batch-Transform-Inference-Job-with-a-fine-tuned-model-using-jsonl)   

HuggingFace Inference DLCsとAmazon SageMaker Python SDKを使用して、Transformersモデルでバッチ変換ジョブを実行します。
このNotebookでは10,000以上のHugging Face Transformersモデルが存在するHuggingFace 🤗 [Hub](https://huggingface.co/models)にあるモデルを使用します。

_**Note: 2021/09時点では日本語処理ライブラリの追加のため、あらかじめコンテナイメージを作成する必要があります。**_

## Run Batch Transform after training a model 

モデルを学習した後、[Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)を使用して、モデルを使った推論を実行できます。Batch Transformでは、S3に保存された推論データに対して、SageMakerがデータのダウンロード、予測の実行、結果のS3へのアップロードを行います。Batch Transformの詳細なドキュメントは[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)にあります。

**HuggingFace estimator**を使用してモデルを学習した場合、`transformer()`メソッドを起動して、トレーニングジョブに基づいてモデルのバッチ変換ジョブを作成することができます。

```python
batch_job = huggingface_estimator.transformer(
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    strategy='SingleRecord')

batch_job.transform(
    data='s3://s3-uri-to-batch-data',
    content_type='application/json',    
    split_type='Line')
```

内容の詳細については、[API docs](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform)をご参照ください。

**このNotebookではモデルの学習は行わず、HuggingFace 🤗 [Hub](https://huggingface.co/models)にあるモデルを使用します。**

## IAM Role

Note: IAMロールに以下の権限があることを確認してください:

- AmazonSageMakerFullAccess
- AmazonS3FullAccess
- AmazonEC2ContainerRegistryFullAccess

ECRへイメージをpushするために、IAMにAmazonEC2ContainerRegistryFullAccessの権限を付与する必要があります。

In [1]:
!pip install "sagemaker>=2.48.0" "datasets==1.11" --upgrade

# Run Batch Transform Inference Job with a fine-tuned model using `jsonl`

In [2]:
import sagemaker
from sagemaker.s3 import S3Uploader, s3_path_join

# get the s3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()

### Create an Amazon ECR registry

In [3]:
import boto3

sess = boto3.Session()

registry_name = 'huggingface-japanese-inference-gpu'
account = boto3.client('sts').get_caller_identity().get('Account')
region = sess.region_name

!aws ecr create-repository --repository-name {registry_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'huggingface-japanese-inference-gpu' already exists in the registry with id '015245611876'


### Build a HuggingFace Docker container and push it to Amazon ECR

Dockerfileは[こちら](https://github.com/aws/deep-learning-containers/blob/master/huggingface/pytorch/inference/docker/1.7/py3/cu110/Dockerfile.gpu)を一部修正し、使用しています。    

変更点

- 16行目: `TRANSFORMERS_VERSION` → `TRANSFORMERS_VERSION=4.6.1`
- 111行目: `transformers[sentencepiece]` → `transformers[ja]`

サンプルはGPUインスタンス用となっており、CPUインスタンス上で推論したい場合は[こちら](https://github.com/aws/deep-learning-containers/blob/master/huggingface/pytorch/inference/docker/1.7/py3/Dockerfile.cpu)を参照してください。

In [4]:
%%time

image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

%cd container_gpu
!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}
%cd ../

/home/ec2-user/SageMaker/sagemaker_nlp_examples/SequenceClassification/pytorch_batch_transform_inference/container_gpu
Sending build context to Docker daemon  9.216kB
Step 1/33 : FROM nvidia/cuda:11.0-cudnn8-runtime-ubuntu18.04
 ---> 848be2582b0a
Step 2/33 : LABEL maintainer="Amazon AI"
 ---> Using cache
 ---> 2576dc84390b
Step 3/33 : LABEL dlc_major_version="1"
 ---> Using cache
 ---> face37200714
Step 4/33 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> 3a07f4ab761b
Step 5/33 : ARG MMS_VERSION=1.1.4
 ---> Using cache
 ---> 70d19697770b
Step 6/33 : ARG PYTHON=python3
 ---> Using cache
 ---> 305df2876244
Step 7/33 : ARG PYTHON_VERSION=3.6.13
 ---> Using cache
 ---> 9d628b75f27f
Step 8/33 : ARG OPEN_MPI_VERSION=4.0.1
 ---> Using cache
 ---> 47d2d2c60bee
Step 9/33 : ARG PT_INFERENCE_URL=https://aws-pytorch-binaries.s3-us-west-2.amazonaws.com/r1.7.1_inference/20210112-183245/c1130f2829b03c0997b9813211a7c0f600fc569a/gpu/torch-1.7.1-cp36-cp36m-m

In [ ]:
image = '015245611876.dkr.ecr.us-west-2.amazonaws.com/huggingface-japanese-inference-gpu:v1'

### Data preparation

Amazon の商品レビューデータセットは[Registry of Open Data on AWS](https://registry.opendata.aws/)で公開されており、 以下からダウンロード可能です。    
このノートブックでは、日本語のデータセットをダウンロードします。

- データセットの概要    
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）    
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [5]:
import urllib.request
import os
import gzip
import shutil


download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

File amazon_review.tsv.gz already exists. Skipped download.
File amazon_review.tsv already exists. Skipped unzip.


In [6]:
import pandas as pd

df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

print(df_pos_neg.shape)
df_pos_neg.head()

(234228, 2)


,star_rating,review_body
0,0,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…
1,0,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…
2,1,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。
3,1,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...
4,1,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。<br />...


In [7]:
# デモ用にサンプリングしてデータを小さくします
df_pos_neg = df_pos_neg.sample(2000, random_state=42)

In [8]:
# テキストのカラム名を'review_body'から'inputs'へ変更してcsvで保存します
df_pos_neg = df_pos_neg.rename(columns={'review_body': 'inputs'})
df_pos_neg.loc[:, 'inputs'].to_csv('review_data.csv', index=False)

### Data preprocessing

先ほど保存した`csv`をバッチ変換ジョブで使用するには、`jsonl`ファイルに変換して、S3にアップロードする必要があります。テキストは複雑な構造をしているので、バッチ変換ジョブでサポートしているのは `jsonl` ファイルだけです。前処理として、改行コード（`<br />`）を削除して、文字数を500文字以内にしています。

バッチ変換ジョブを行う際には、`inputs`のトークン数が`max_length`に収まっていることを確認する必要があります。ここでは簡便的に最大文字数を500文字にして`max_length`内に収めています（これから使用するモデルの`max_length`は512）。

In [9]:
import csv
import json

# datset files
dataset_csv_file="review_data.csv"
dataset_jsonl_file="review_data.jsonl"

In [10]:
with open(dataset_csv_file, "r+") as infile, open(dataset_jsonl_file, "w+") as outfile:
    reader = csv.DictReader(infile, )
    for row in reader:
        row["inputs"] = row["inputs"].replace("<br />","")[:500]
        json.dump(row, outfile, ensure_ascii=False)
        outfile.write('\n')

In [11]:
# uploads a given file to S3.
input_s3_path = s3_path_join("s3://", sagemaker_session_bucket, "batch_transform/input")
output_s3_path = s3_path_join("s3://", sagemaker_session_bucket, "batch_transform/output")
s3_file_uri = S3Uploader.upload(dataset_jsonl_file, input_s3_path)

print(f"{dataset_jsonl_file} uploaded to {s3_file_uri}")

review_data.jsonl uploaded to s3://sagemaker-us-west-2-015245611876/batch_transform/input/review_data.jsonl


`review_data.jsonl`は以下のようになります。

```json
{"inputs": "確かに映像はきれいですが、話の内容的には、「……」という感じです。いかにもワイヤーというのが、見ていて飽き飽きしてしまいました。"}
{"inputs": "天使の街で、死の天使が微笑んだ。1940年代後半ロサンゼルスは、華やかな顔のその裏で暴力がはびこり裏社会の頂点に立った男が、悪の限りを尽くしていた。その男の名はミッキー・コーエン、司法 , 警察までもが彼の手中にあり、腐敗しきっていた。そんな時市警察の異端児たちが密命を受け、街を取り戻すため、命を懸けて裏社会に戦いを挑んだ。人の装いや暮らし , 街並み、そして街を流す車に時代の色と香りが溢れます。日が沈むと・・・夜な夜な人々が集う、華やかでそして危険な社交場。昼と夜、表と裏、光と影、裏社会の勢いがそんな垣根も越えて行く時代。でも、喰うか喰われるか , 生きるか死ぬかの (筈の) 戦いが始まると、少々拍子抜け。ギリギリした緊張感や、胃液が上がって来る様な緊迫感に欠ける感じがする。その点が一番惜しい。金と血の匂いが好きなミッキー・コーエンを、ショーン・ペンがどんぴしゃりと演じています。ライアン・ゴズリングは、あの時代背景の中で映えまくっています。当時の装いを見事に着こなし、荒んで見せても、甘く繊細な雰囲気が漂います。何を期待したのか、それによって印象がだいぶ変わるかもしれません。私自身肩透"}
{"inputs": "2008年にリリースされた、USのヘヴィロック・バンドの3rdアルバム。日本ではあまり知られていないかもしれませんが、本国アメリカでは知名度のあるバンドです。このバンドの主役はヴォーカルのブレント・スミスの熱い歌唱だと思います。男臭いエモーショナルな歌声で、とても良い声をしています。曲は、疾走感のあるハードなロックナンバーから歌声で聴かせるバラードまであり、それぞれ曲の質が高いと思います。キャッチーさがあって聴きやすいのも良いです。1曲目の「Devour」は繰り返されるフレーズとサビが印象的なノリの良いハードなナンバーで、私は初めて聴いた時はDISTURBEDかと思いました・・・。久しぶりに聴いたのですが、今聴いても良いと思うアルバムでした。"}
{"inputs": "まさにストリートレース!!この映画を越える作品は無いんじゃないかな、何よりスピード感 最初のドラッグレース シーンは迫力満点、後半でエクリプスがNosを噴射したシーンは画面が揺れ 画面効果が凄い、さすがロブ・コーエン監督!!車はどちらかと言うと 見た目はスポコンで 日本車がメイン!!インテグラ シビック JZA80スープラ FD3S S14シルビア R33 GT-R エクリプス日本からは Bomexなど有名ショップも協力していて出来がGood!!!最後には1970 Dodge Chargerが出演 最後までスピード感があり ストーリー性も完璧です。"}
{"inputs": "各国版が出ていて、共通のテキストとしては最適だと思います。ただ、内容がちょっと古いことと、（今時ハンサムはない）出てくる人の名前がとても読みにくい。"}
....
```

## Create Inference Transformer to run the batch job

_これは実験的な機能で、エンドポイントが作成された後にモデルが読み込まれるようになっています。これにより、モデルが10GBを超える場合などでエラーが発生する可能性があります_

🤗 HubからSageMakerにモデルを直接デプロイするには、`HuggingFaceModel`の作成時に2つの環境変数を定義する必要があります:

- `HF_MODEL_ID`: SageMakerエンドポイントを作成する際に、[huggingface.co/models](http://huggingface.co/models) から自動的にロードされるモデルIDを定義します。🤗 Hubは10,000以上のモデルを提供しており、この環境変数で利用できます。

- `HF_TASK`: 使用する🤗 Transformersのパイプラインのタスクを定義します。タスクの完全なリストは [ここ](https://huggingface.co/transformers/main_classes/pipelines.html) にあります。

このサンプルでは、🤗 Hubから https://huggingface.co/abhishek/autonlp-japanese-sentiment-59363 をデプロイします。

In [12]:
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'abhishek/autonlp-japanese-sentiment-59363', # model_id from hf.co/models
  'HF_TASK':'text-classification' # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=image,
    env=hub,
    role=role, # iam role with permissions to create an Endpoint
    #transformers_version="4.6", # transformers version used
    #pytorch_version="1.7", # pytorch version used
    #py_version="py36", # python version of the DLC
)

# create Transformer to run our batch job
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    output_path=output_s3_path, # we are using the same s3 path to save the output with the input
    strategy='SingleRecord')

In [13]:
# starts batch transform job and uses s3 data as input
batch_job.transform(
    data=s3_file_uri,
    content_type='application/json',    
    split_type='Line')

......................................This is an experimental beta features, which allows downloading model from the Hugging Face Hub on start up. It loads the model defined in the env var `HF_MODEL_ID`
#015Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 1.11k/1.11k [00:00<00:00, 827kB/s]
#015Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 843/843 [00:00<00:00, 604kB/s]
#015Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]#015Downloading:   1%|          | 3.13M/445M [00:00<00:14, 31.2MB/s]#015Downloading:   2%|▏         | 8.22M/445M [00:00<00:10, 42.8MB/s]#015Downloading:   3%|▎         | 13.6M/445M [00:00<00:09, 47.9MB/s]#015Downloading:   4%|▍         | 18.9M/445M [00:00<00:08, 49.8MB/s]#015Downloading:   5%|▌         | 24.3M/445M [00:00<00:08, 51.3MB/s]#015Downloading:   7%|▋         | 29.7M/445M [00:00<00:07, 52.4MB/s]#015Downloading:   8%|▊         | 35.1M/445M [00:00<00:07, 52.8MB/s]#015Downlo

2021-09-22 05:19:24,910 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 434
2021-09-22 05:19:24,910 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 5348
2021-09-22 05:19:25,003 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.044345855712890625 ms
2021-09-22 05:19:25,003 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 27.95267105102539 ms
2021-09-22 05:19:25,003 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 29
2021-09-22 05:19:25,003 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.04553794860839844 ms
2021-09-22 05:19:25,003 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 30
2021-

2021-09-22 05:19:32,009 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.030755996704101562 ms
2021-09-22 05:19:32,009 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 24
2021-09-22 05:19:32,009 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 23.723840713500977 ms
2021-09-22 05:19:32,009 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.029087066650390625 ms
2021-09-22 05:19:32,009 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 24
2021-09-22 05:19:32,031 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.020742416381835938 ms
2021-09-22 05:19:32,031 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict 

2021-09-22 05:19:34,553 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.021457672119140625 ms
2021-09-22 05:19:34,553 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 18.935203552246094 ms
2021-09-22 05:19:34,553 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.028133392333984375 ms
2021-09-22 05:19:34,553 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 19
2021-09-22 05:19:34,553 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 20
2021-09-22 05:19:34,574 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.020742416381835938 ms
2021-09-22 05:19:34,574 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict 

2021-09-22 05:19:41,105 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 20
2021-09-22 05:19:41,127 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.022649765014648438 ms
2021-09-22 05:19:41,127 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 20.14756202697754 ms
2021-09-22 05:19:41,127 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.028848648071289062 ms
2021-09-22 05:19:41,127 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 21
2021-09-22 05:19:41,127 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 21
2021-09-22 05:19:41,158 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 29
2021-09

...

2021-09-22 05:19:51,727 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 20
2021-09-22 05:19:51,747 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.021219253540039062 ms
2021-09-22 05:19:51,747 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 18.639087677001953 ms
2021-09-22 05:19:51,747 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.028133392333984375 ms
2021-09-22 05:19:51,661 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 20
2021-09-22 05:19:51,683 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.021696090698242188 ms
2021-09-22 05:19:51,683 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Pr

2021-09-22 05:19:54,629 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 20.341873168945312 ms
2021-09-22 05:19:54,629 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.030755996704101562 ms
2021-09-22 05:19:54,630 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 21
2021-09-22 05:19:54,630 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 21
2021-09-22 05:19:54,651 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.02193450927734375 ms
2021-09-22 05:19:54,651 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 19.04439926147461 ms
2021-09-22 05:19:54,651 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess tim

2021-09-22 05:20:01,137 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 21
2021-09-22 05:20:01,158 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.020503997802734375 ms
2021-09-22 05:20:01,158 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 18.9054012298584 ms
2021-09-22 05:20:01,158 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.0286102294921875 ms
2021-09-22 05:20:01,158 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 19
2021-09-22 05:20:01,158 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 20
2021-09-22 05:20:01,179 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.020980834

2021-09-22 05:20:05,111 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.030040740966796875 ms
2021-09-22 05:20:05,111 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 20
2021-09-22 05:20:05,021 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 21.613121032714844 ms
2021-09-22 05:20:05,021 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.03361701965332031 ms
2021-09-22 05:20:05,022 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 23
2021-09-22 05:20:05,022 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 23
2021-09-22 05:20:05,043 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.02217292785

2021-09-22 05:20:09,801 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 21
2021-09-22 05:20:09,801 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 21
2021-09-22 05:20:09,801 [INFO ] W-9000-abhishek__autonlp-japanes ACCESS_LOG - /169.254.255.130:49858 "POST /invocations HTTP/1.1" 200 21
2021-09-22 05:20:09,823 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Preprocess time - 0.021457672119140625 ms
2021-09-22 05:20:09,823 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Predict time - 20.47586441040039 ms
2021-09-22 05:20:09,823 [INFO ] W-abhishek__autonlp-japanes-1-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Postprocess time - 0.030755996704101562 ms
2021-09-22 05:20:09,823 [INFO ] W-9000-abhishek__autonlp-japanes com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 21
2021-09-22 05

## Download-predict-file-from-s3

In [14]:
from sagemaker.s3 import S3Downloader
from ast import literal_eval

# creating s3 uri for result file -> input file + .out
output_file = f"{dataset_jsonl_file}.out"
output_path = s3_path_join(output_s3_path,output_file)

# download file
S3Downloader.download(output_path,'.')

In [15]:
batch_transform_result = []
with open(output_file) as f:
    for line in f:
        # converts jsonline array to normal array
        line = "[" + line.replace("[","").replace("]",",") + "]"
        batch_transform_result = literal_eval(line) 
        
# print results 
print(batch_transform_result[:5])

[{'label': 'negative', 'score': 0.997538149356842}, {'label': 'positive', 'score': 0.9994381666183472}, {'label': 'positive', 'score': 0.9996545910835266}, {'label': 'positive', 'score': 0.999487042427063}, {'label': 'positive', 'score': 0.9977704286575317}]


In [16]:
df_pos_neg.head()

,star_rating,inputs
239616,0,確かに映像はきれいですが、<br />話の内容的には、「……」という感じです。<br />い...
219139,1,天使の街で、死の天使が微笑んだ。<br />1940年代後半ロサンゼルスは、華やかな顔のその...
60193,1,2008年にリリースされた、USのヘヴィロック・バンドの3rdアルバム。<br /><br ...
51352,1,まさにストリートレース!!この映画を越える作品は無いんじゃないかな、何よりスピード感 最初の...
92214,1,各国版が出ていて、共通のテキストとしては最適だと思います。ただ、内容がちょっと古いことと、（...
